## Group No 048

## Group Member Names:
1. Bhavesh Pandey (2020SC04857)
2. Nimish Pandey (2020SC04328)
3. Harsh Vashisht (2020SC04565)



# 1. Business Understanding
 
Students are expected to identify a classification problem of your choice. You have to detail the Business Understanding part of your problem under this heading which basically addresses the following questions.
 
   1. What is the problem that you are trying to solve?
   2. What data do you need to answer the above problem?
   3. What are the different sources of data?    
   4. What kind of analytics task are you performing?



# 2. Data Acquisition
 
For the problem identified by you, students have to find the data source themselves which should be a website which has the required data in it. You have to write Python crawler code to scrape data from the respective website rather than downloading ready-made dataset as such from sources like Kaggle etc. 

(Data downloaded from website like Kaggle will be awarded negative marks.)

## 2.1 Code for scraping data from website



In [ ]:
# pip install pandas_datareader
# pip install bs4
# pip install mlxtend

In [ ]:
##---------Type the code below this line------------------##
import os
import re
import requests
import csv
import shutil
import numpy as np
import pandas as pd
import bs4 as bs
import pandas_datareader as web
import itertools
from itertools import chain
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import date
from datetime import timedelta
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

In [ ]:
final_tickers=[]

def contains(s):
    return not re.match(r'[^.]+$', s)

def nifty50_tickers():
    link = 'https://en.wikipedia.org/wiki/NIFTY_50'
    response = requests.get(link)
    soup = bs.BeautifulSoup(response.text, 'lxml')
    table = soup.find('table', {'class' : 'wikitable sortable'}, {'id' : 'constituents'})
    tickers=[]
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[1].text[:]
        tickers.append(ticker)
    for i in tickers:
        if contains(i) != True:
            final_tickers.append(i)
            
    return final_tickers

nifty50_tickers()

string=".NS"
new_tickerlist = [x + string for x in final_tickers]

start = date.today() - timedelta(days=2190)
end = date.today()

if not os.path.exists('Companies'):
    os.makedirs('Companies')

for ticker in new_tickerlist:
    if not os.path.exists('Companies/{}.csv'.format(ticker)):
        print("{} - Fetching Data... ".format(ticker))
        df = web.DataReader(ticker, 'yahoo', start, end)
        df.to_csv('Companies/{}.csv'.format(ticker))
    else:
        print("{} - Already Fetched.!".format(ticker))

## 2.2 Code for converting the above scraped data into a dataframe

In [ ]:
##---------Type the code below this line------------------##
main_df = pd.DataFrame()

for ticker in new_tickerlist:
    df = pd.read_csv('Companies/{}.csv'.format(ticker))
    df.set_index('Date', inplace=True)
    df.rename(columns = {'Close': ticker}, inplace=True)
    df.drop(columns = ['Open', 'High', 'Low', 'Adj Close', 'Volume'], inplace=True)
    if main_df.empty:
        main_df = df
    else:
        main_df = main_df.join(df, how='outer')
        
main_df = main_df.reset_index()
main_df.set_index('Date', inplace=True)
Dataframe = pd.concat([main_df.tail(1)])

high = Dataframe.copy()
open1 = Dataframe.copy()
low = Dataframe.copy()
close = Dataframe.copy()

high.loc[:] = np.nan
open1.loc[:] = np.nan
low.loc[:] = np.nan
close.loc[:] = np.nan

Datelist=[]

if not os.path.exists("Companies_data"):
    os.makedirs("Companies_data")

for ticker in new_tickerlist:
    with open("Companies_data/{}.csv".format(ticker), 'w') as f:
        pass
    df = pd.read_csv('Companies/{}.csv'.format(ticker))
    data1 = pd.concat([df.tail(2)])
    data1.drop(df.tail(1).index,inplace=True)
    data1.to_csv("Companies_data\{}.csv".format(ticker), header=True, index=False)
    with open("Companies_data\{}.csv".format(ticker)) as f:
        companies_csv = csv.DictReader(f)
        for row in companies_csv:
            Date = row['Date']
            HighPrice = float(float(row['High']))
            OpenPrice = float(float(row['Open']))
            LowPrice = float(float(row['Low']))
            ClosePrice = float(float(row['Close']))
        H = HighPrice
        O = OpenPrice
        L = LowPrice
        C = ClosePrice
    Datelist.append(Date)
    high[ticker].fillna(H, inplace=True)
    open1[ticker].fillna(O, inplace=True)
    low[ticker].fillna(L, inplace=True)
    close[ticker].fillna(C, inplace=True)
    
newdata = pd.DataFrame(columns=['Companies', '200_Moving_AVG', '50_Moving_AVG'])
newdata['Companies'] = new_tickerlist
mva50all, mva200all = ([] for i in range(2))

for tickers in new_tickerlist:
    df = pd.read_csv('Companies/{}.csv'.format(tickers))
    mva50 = round(df['Close'].rolling(window=50, min_periods=0).mean(), 3)
    mva200 = round(df['Close'].rolling(window=200, min_periods=0).mean(), 3)
    df['50_mVA'] = mva50
    df['200_mvA'] = mva200
    df.to_csv('Companies/{}.csv'.format(tickers), index=False)       
    mva50all.append(mva50.iloc[-1])
    mva200all.append(mva200.iloc[-1])
    
newdata['200_Moving_AVG'] = mva200all
newdata['50_Moving_AVG'] = mva50all

high1,low1,open2,close1 = ([] for i in range(4))
    
for (colname, colval) in low.iteritems():
    low1.append(round(float(colval), 3))
    
for (colname, colval) in open1.iteritems():
    open2.append(round(float(colval), 3))

for (colname, colval) in high.iteritems():
    high1.append(round(float(colval), 3))
    
for (colname, colval) in close.iteritems():
    close1.append(round(float(colval), 3))

## 2.3 Confirm the data has been correctly by displaying the first 5 and last 5 records.

In [ ]:
##---------Type the code below this line------------------##
for tickers in new_tickerlist:
    df = pd.read_csv('Companies/{}.csv'.format(ticker))
    print(df.head(5))

In [ ]:
for tickers in new_tickerlist:
    df = pd.read_csv('Companies/{}.csv'.format(ticker))
    print(df.tail(5))

## 2.4 Display the column headings, statistical information, description and statistical summary of the data.

In [ ]:
##---------Type the code below this line------------------##
for tickers in new_tickerlist:
    df = pd.read_csv('Companies/{}.csv'.format(ticker))
    print(df.describe())

## 2.5 Write your observations from the above. 
1. Size of the dataset
2. What type of data attributes are there?
3. Is there any null data that has to be cleaned?


# 3. Data Preparation

## 3.1 Display how many unique values are present in each attribute


In [ ]:
##---------Type the code below this line------------------##
for tickers in new_tickerlist:
    df = pd.read_csv('Companies/{}.csv'.format(tickers))
    print(df.nunique())

## 3.2 Check for the presence of duplicate data, identify the attributes with duplicate data, report the attributes. Mention the method adopted to remove duplicate data if present. Report the results again.

In [ ]:
##---------Type the code below this line------------------##
for ticker in new_tickerlist:
    df = pd.read_csv('Companies/{}.csv'.format(ticker))
    print(df.duplicated())

## 3.3 Show whether there are any missing values in each attribute. Report the same.


In [ ]:
##---------Type the code below this line------------------##
for ticker in new_tickerlist:
    df = pd.read_csv('Companies/{}.csv'.format(ticker))
    print(df.isnull())

## 3.4 Clean the missing data using any imputation technique, mention the method used and again report the change after cleaning the data.

## 3.5 Check if all the attributes are following the same format and are consistent. If not, report all such attributes and what inconsistencies are present.

In [ ]:
##---------Type the code below this line------------------##
for ticker in new_tickerlist:
    df = pd.read_csv('Companies/{}.csv'.format(ticker))
    print(df.info())

## 3.6 Correct the data if there are inconsistencies from 3.5. Report or print the data after correction.

## 3.7 Identify the target variables.


In [ ]:
##---------Type the code below this line------------------##

if not os.path.exists('DuplicateData'):
    os.makedirs('DuplicateData')
    
for ticker in new_tickerlist:
    df = pd.read_csv('Companies/{}.csv'.format(ticker), index_col=0)
    days = 1
    df['pred_cp'] = df['Adj Close'].shift(-days)
    df['High'] = df['High'].shift(-days)
    df['Low'] = df['Low'].shift(-days)
    df['Open'] = df['Open'].shift(-days)
    df['Close'] = df['Close'].shift(-days)
    df.to_csv('Companies/{}.csv'.format(ticker), header=True)

## 3.8 Separate the data front the target such that the dataset is in the form of (X,y) or (Features, Label)

In [ ]:
##---------Type the code below this line------------------##
for ticker in new_tickerlist:
    df = pd.read_csv('Companies/{}.csv'.format(ticker))
    df.dropna(inplace=True)
    X = np.array(df[['Open', 'High', 'Low', 'Close', '50_mVA', '200_mvA']])
    y = np.array(df['pred_cp'])

## 3.9 Discretize the target variable or perform one-hot encoding on the target or any other as and if required.


# 4. Data Exploration using various plots

## 4.1 Scatter plot of each attribute with the target.

In [ ]:
##---------Type the code below this line------------------##
print(plt.scatter(X.T[0], y))
print(plt.scatter(X.T[1], y))
print(plt.scatter(X.T[2], y))
print(plt.scatter(X.T[3], y))
print(plt.scatter(X.T[4], y))
print(plt.scatter(X.T[5], y))
# Scatter plot of the last dataset.

## 4.2 Pair plot of each attribute to identify the linear relationships among the attributes.


In [ ]:
##---------Type the code below this line------------------##
for ticker in new_tickerlist:
    pp = pd.read_csv('Companies/{}.csv'.format(ticker))
print(sns.pairplot(pp))
# Pair Plot of the last dataset.

## 4.3 Regression plots to identify the linear relationship between each attribute with the target variable. 


In [ ]:
for ticker in new_tickerlist:
    rg1 = pd.read_csv('Companies/{}.csv'.format(ticker))
print(sns.regplot(x='High', y='pred_cp', data=rg1))
# Regression Plot of the last dataset

In [ ]:
for ticker in new_tickerlist:
    rg2 = pd.read_csv('Companies/{}.csv'.format(ticker))
print(sns.regplot(x='Open', y='pred_cp', data=rg2))
# Regression Plot of the last dataset

In [ ]:
for ticker in new_tickerlist:
    rg3 = pd.read_csv('Companies/{}.csv'.format(ticker))
print(sns.regplot(x='Close', y='pred_cp', data=rg3))
# Regression Plot of the last dataset

In [ ]:
for ticker in new_tickerlist:
    rg4 = pd.read_csv('Companies/{}.csv'.format(ticker))
print(sns.regplot(x='Low', y='pred_cp', data=rg4))
# Regression Plot of the last dataset

In [ ]:
for ticker in new_tickerlist:
    rg5 = pd.read_csv('Companies/{}.csv'.format(ticker))
print(sns.regplot(x='Adj Close', y='pred_cp', data=rg5))
# Regression Plot of the last dataset

In [ ]:
for ticker in new_tickerlist:
    rg6 = pd.read_csv('Companies/{}.csv'.format(ticker))
print(sns.regplot(x='Volume', y='pred_cp', data=rg6))
# Regression Plot of the last dataset

## 4.4 Can any other plot help to identify the optimal set of attributes that can be used for classification. The plot will be based on linear or nonlinear separations. If there is/are such plots, name them, explain why you think they can be helpful in the task and perform the plot as well.


# 5. Data Wrangling



## 5.1 Display correlation heatmap of each attribute against the target and report which features are significant.

In [ ]:
##---------Type the code below this line------------------##
for ticker in new_tickerlist:
    hm = pd.read_csv('Companies/{}.csv'.format(ticker), index_col=0)
print(sns.heatmap(hm))
# Heatmap of the last dataset.

## 5.2 Univariate Filters – Identify top 5 significant features by evaluating each feature independently with respect to the target variable by exploring 
1. Mutual Information (Information Gain)
2. Gini index
3. Gain Ratio
4. Chi-Squared test
5. Fisher Score
(From the above 5 you are required to use only any <b>three</b>) 

Write your observations from the results of each method and report the top 5 significant features for each of the above methods. Also plot a graph of significant features for each of them for better visualization.

## 5.3 Train a “DecisionTreeClassifier” on the entire data and use the classifier to extract the top 5 significant features. Plot graph of significant features for better visualization.

## 5.4 Using "mlxtend" library perform SequentialFeatureSelector to identify top 5 features.

In [ ]:
##---------Type the code below this line------------------##
predictive_values,confidence1,kfeature,kscore = ([] for i in range(4))

for ticker in new_tickerlist:
    df = pd.read_csv('Companies/{}.csv'.format(ticker), index_col=0)
    df.dropna(inplace=True)
    
    X = np.array(df[['Open', 'High', 'Low', 'Close', '50_mVA', '200_mvA']])
    y = np.array(df['pred_cp'])

    lab_enc = preprocessing.LabelEncoder()
    encoded = lab_enc.fit_transform(y)
    knn = KNeighborsClassifier(n_neighbors=2)

    sfs = SFS(knn, k_features=5, forward=True, floating=False, verbose=2, scoring='accuracy', cv=0)

    feature_names = ('Open', 'High', 'Low', 'Close', '50_mVA', '200mVA')
    sfs = sfs.fit(X, encoded, custom_feature_names=feature_names)
    kfeature.append(sfs.k_feature_idx_)
    kscore.append(sfs.k_score_)
    
    X = preprocessing.scale(X)
    
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=1)
    clf = LinearRegression()
    clf.fit(X_train, y_train)
    confidence = clf.score(X_test, y_test)
    confidence1.append(confidence)

    X = X[:-days]
    X_new = X[-days:]
    prediction = clf.predict(X_new)
    
    predictive_values.append(float(np.ma.round(prediction, decimals=3)))

In [ ]:
final_dataframe = pd.DataFrame(columns=['Date', 
                                         'Companies', 
                                         'Open', 
                                         'High', 
                                         'Low',
                                         'Close',
                                         '50d Moving AVG',
                                         '200d Moving AVG',
                                         'Predictive Close',
                                         'Confidence',
                                         'k_feature',
                                         'k_score'])

final_dataframe['Date'] = Datelist
final_dataframe['Companies'] = new_tickerlist
final_dataframe['Open'] = open2
final_dataframe['High'] = high1
final_dataframe['Low'] = low1
final_dataframe['Close'] = close1
final_dataframe['50d Moving AVG'] = mva50all
final_dataframe['200d Moving AVG'] = mva200all
final_dataframe['Predictive Close'] = predictive_values
final_dataframe['Confidence'] = confidence1
final_dataframe['k_feature'] = kfeature
final_dataframe['k_score'] = kscore

final_dataframe

In [ ]:
shutil.rmtree('Companies')
shutil.rmtree('Companies_data')

## 5.5 Conclude the top 3 significant features with necessary justifications.

--------------Type the answers below this line--------------

In section 5.4 top 5 features has been selected which shows that for maximum number of companies feature set (0, 1, 3, 4, 5) is most useful. As per this following are the top 3 significant features:

1. Open
2. High
3. Close

